# Chương 9: Không gian vector (I)
File enwiki-20150112-400-r10-105752.txt.bz2 là file nén dạng bzip2 của 105752 file text được lấy mẫu ngẫu nhiên (tỷ lệ 1/10) từ các bài báo trên Wikipedia có trên 400 từ. Các bài báo trên Wikipedia được lấy vào ngày 12 tháng 1 năm 2015. Sử dụng dữ liệu file này làm corpus để học các vector thể hiện ý nghĩa của các từ. Trong nửa đầu của chương 9, bạn được yêu cầu trích xuất các context của các từ, trích xuất đặc trưng, và dùng phương pháp PCA để giảm bớt số chiều của dữ liệu. Nửa sau của chương 9 yêu cầu bạn tính độ tương tự của các từ sử dụng các word vectors đã học từ corpus.

Chú ý, bài 83 yêu cầu 7GB memory. Trong trường hợp lượng memory của bạn không đủ, bạn cần có các phương pháp xử lý thích hợp hoặc sử dụng sample 1/100 của dữ liệu trong file enwiki-20150112-400-r10-105752.txt.bz2.

In [7]:
%cd /content/drive/My Drive/github_repos/practice_100_nlp_exercises/

/content/drive/My Drive/github_repos/practice_100_nlp_exercises


# Download data

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!wget http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2

--2020-04-18 15:55:04--  http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221903910 (212M) [application/x-bzip2]
Saving to: ‘enwiki-20150112-400-r10-105752.txt.bz2’

enwiki-20150112-400 100%[===================>] 211.62M  11.2MB/s    in 19s     

2020-04-18 15:55:24 (11.1 MB/s) - ‘enwiki-20150112-400-r10-105752.txt.bz2’ saved [221903910/221903910]



In [0]:
!bzip2 -d enwiki-20150112-400-r10-105752.txt.bz2

# 80. Tiền xử lý dữ liệu
Sử dụng khoảng trắng là ký tự ngăn cách để tokenize các từ trong các câu. Phương pháp này có nhược điểm là các từ thu được sẽ còn các ký tự đặc biệt như dấu câu, hoặc dấu ngoặc. Vì thế sau khi tokenize các từ trong corpus, tiến hành các xử lý sau đây.

* Xoá các ký tự đặc biệt xuất hiện ở đầu và cuối các từ: .,!?;:()[]'"
* Xoá các từ chỉ gồm ký tự trắng

Sau khi tiền xử lý dữ liệu, lưu file dữ liệu gồm danh sách các từ cách nhau bởi khoảng trắng.


In [0]:
!head enwiki-20150112-400-r10-105752.txt

Anarchism

Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions, but that several authors have defined as more specific institutions based on non-hierarchical free associations. Anarchism holds the state to be undesirable, unnecessary, or harmful. While anti-statism is central, anarchism entails opposing authority or hierarchical organisation in the conduct of human relations, including, but not limited to, the state system.
As a subtle and anti-dogmatic philosophy, anarchism draws on many currents of thought and strategy. Anarchism does not offer a fixed body of doctrine from a single particular world view, instead fluxing and flowing as a philosophy. There are many types and traditions of anarchism, not all of which are mutually exclusive. Anarchist schools of thought can differ fundamentally, supporting anything from extreme individualism to complete collectivism. Strains of anarchism have often been divided int

In [0]:
with open('enwiki-20150112-400-r10-105752.txt', 'r') as f:
    lines = f.readlines()

len(lines)

2875326

In [0]:
import re

with open('enwiki_tokenized.txt', 'w') as f:
    for line in lines:
        line = line.strip()

        if line == '':
            continue

        # Remove special characters
        line = re.sub(r'[\.,!\?;:\(\)\[\]\'\"]', '', line)

        # Split by space
        line = line.split(' ')

        # Remove all empty elements
        line = list(filter(lambda a: a != '', line)) # Maybe use filter is faster than other methods

        # Join list by space
        line = ' '.join(line)

        # Write to file
        f.write(line + '\n')

In [0]:
!head 'enwiki_tokenized.txt'

Anarchism
Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories of

In [0]:
# Count number of lines (Linux)
!wc -l 'enwiki_tokenized.txt'

2557968 enwiki_tokenized.txt


In [0]:
# Count number of words (Linux)
!wc -w 'enwiki_tokenized.txt'

120307584 enwiki_tokenized.txt


# 81. Xử lý tên các nước tạo thành từ các compound words
Trong tiếng Anh, nhiều từ cạnh nhau có thể tạo thành một từ có ý nghĩa. Ví dụ, hợp chủng quốc Hoa Kỳ là "United States", vương quốc Anh là "United Kingdom". Nếu chỉ dùng các "United", "State", hay "Kingdom" như các từ riêng lẻ, ý nghĩa của các từ này sẽ nhập nhằng. Vì thế trong khi tiền xử lý dữ liệu, ta cần xác định các từ ghép này. Đoán nhận các từ ghép là một bài toán khó, nên ở đây ta chỉ đoán nhận các từ ghép là tên của các nước.

Trước hết, download danh sách tên của các nước trên Internet. Dùng danh sách tên các nước này để đoán nhận các từ ghép là tên nước trong dữ liệu sử dụng ở bài 80, sau đó biến đổi các ký tự spaces thành ký tự underscore (_) để nối các từ thành phần. Ví dụ "United States" sẽ trở thành "United_States", "Isle of Man" sẽ trở thành "Isle_of_Man."

In [0]:
# Download list country names from github
!wget -O country_names.txt https://raw.githubusercontent.com/datasets/country-list/master/data.csv

--2020-04-18 16:58:46--  https://raw.githubusercontent.com/datasets/country-list/master/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3870 (3.8K) [text/plain]
Saving to: ‘country_names.txt’

country_names.txt   100%[===================>]   3.78K  --.-KB/s    in 0s      

2020-04-18 16:58:46 (13.0 MB/s) - ‘country_names.txt’ saved [3870/3870]



In [0]:
# Get names only
import pandas as pd

country_name_df = pd.read_csv('country_names.txt')

country_name_df['Name'].to_csv('country_names.txt', index=False, header=False)

In [0]:
!head country_names.txt

Afghanistan
Åland Islands
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antarctica
Antigua and Barbuda


In [0]:
# Get country names which have more than 2 words
list_country_names = [re.sub('[,\"\n\'\(\)\.]', '', s) for s in open('country_names.txt', 'r').readlines()]

country_names_more_than_2_words = [s for s in list_country_names if len(s.split()) > 1]

In [0]:
# Create compound words
def insert_underscore(matchobj):
    return '' + matchobj.group(0).replace(' ', '_')

content = open('enwiki_tokenized.txt', 'r').read()

content = re.sub("(%s)" % "|".join(country_names_more_than_2_words), insert_underscore, content)

with open('enwiki_tokenized_compound_words.txt', 'w') as f:
    f.write(content)

In [0]:
# To display word has _
!grep -oh "\w*_\w*" enwiki_tokenized_compound_words.txt

# 82. Trích xuất context
Sử dụng corpus được tạo ra trong bài tập 81, trích xuất context của tất cả các từ xuất hiện trong corpus. Context c của mỗi từ t trong dữ liệu sẽ cặp với từ t và xuất ra theo định dạng: các thông tin cách nhau bởi ký tự tab. Context của mỗi từ t được định nghĩa như sau:

* Trích xuất các từ ở trước và sau của t với kích thước cửa sổ là d (chú ý context words của t sẽ không bao gồm bản thân của từ t)
* Với mỗi từ t, kích thước của context (window size) d sẽ được chọn ngẫu nhiên trong tập {1, 2, 3, 4, 5}.

In [0]:
# Read file
with open('enwiki_tokenized_compound_words.txt', 'r') as f:
    lines = f.readlines()

from random import randint # for random window size


with open('word_context.txt', 'w') as f:
    for line in lines:
        list_word = line.strip().split()

        n_word = len(list_word)

        for idx,word in enumerate(list_word):
            window_size = randint(1, 5)

            list_context = list_word[max(0, idx - window_size):idx] + list_word[
                                                                                min(idx+1, n_word): min(idx+1+window_size, n_word)]

            for context in list_context:
                f.write(word + '\t' + context + '\n')

In [9]:
!head word_context.txt

Anarchism	is
Anarchism	a
is	Anarchism
is	a
is	political
is	philosophy
is	that
is	advocates
a	Anarchism
a	is


# 83. Tính tần xuất xuất hiện của từ/context
Sử dụng đầu ra của bài 82, tính phân bố xuất hiện và các hằng số sau:

* f(t,c): là số lần đồng xuất hiện của từ t và context word c.
* f(t,*): số lần xuất hiện của từ t.
* f(*,c): số lần xuất hiện của context word c.
* N: Tổng số lần xuất hiện của từ và các context word (hằng số).

In [0]:
# N
!wc -l word_context.txt

120291996 word_context.txt


In [0]:
int(120291996/100)

1202919

In [0]:
# Create new file with fewer lines
!head --lines 1202919 word_context.txt > word_context_fewer.txt

In [0]:
# Read all data
import pandas as pd
df = pd.read_csv('word_context_fewer.txt', delimiter='\t', header=None)

In [0]:
# Freq of all
all_freq = df.groupby(df.columns.tolist(),as_index=False).size().reset_index(name='freq')

all_freq

,0,1,freq
0,#,a few characters such as & on the FIGS layer,1
1,#1,Arthur Park by Summer hit on the Billboard Dan...,1
2,#1,Detective Comics Detective Comics,1
3,#1,Donna Troy and the,1
4,#1,Fun The Big Comic Magazine with a cover date of,1
...,...,...,...
1179262,陰陽家/阴阳家,or the School of Yin-yang Yīnyángjiā Yin-yang-...,1
1179263,麦星,Mugi-boshi meaning,1
1179264,주작,Japanese and Ju-jak Hanja朱雀 in Korean,1
1179265,ﬁnal,triangulation paired with a iterative triangle...,1


In [0]:
# f(t,c) > 30
all_freq[all_freq['freq'] > 30]

,0,1,freq
119903,In,the,55
338955,as,known the,37
341755,as,such the,70
492851,end,the of,53
743018,number,a of,36
743661,number,the of,36
764908,of,end the,34
775462,of,one the,113
776277,of,part the,64
813625,order,in to,32


In [0]:
# f(t, *)
word_counts = df.iloc[:,0].value_counts()

print('Number of words: ', len(word_counts))

print('Top 10 words that appear the most')
word_counts.head(10)

Number of words:  78426
Top 10 words that appear the most


the    78557
of     46246
and    36298
in     27641
to     26834
a      24002
is     14406
The    11612
as     11119
was     9939
Name: 0, dtype: int64

In [0]:
# f(*, c)
context_counts = df.iloc[:,1].value_counts()

print('Number of context: ', len(context_counts))

print('Top 10 context that appear the most')

context_counts.head(10)

Number of context:  1124726
Top 10 context that appear the most


the of     2716
the and     638
a of        614
to the      516
the         487
of and      440
and         339
of          338
The of      316
and the     307
Name: 1, dtype: int64

# 84. 